**Chapter 1: Linear Programming**

***Developed by Anh Phuong Ngo***

# Question 1

## Problem description
A fabric company produces 4 sizes of product: small, medium, large and extra large. These product sizes can be produced on any one of three laser machine types: A, B, and C. The productivity per hour per meter of each machine type for each product type is shown as follow:

<table>
<thead>
  <tr>
    <th></th>
    <th>A</th>
    <th>B</th>
    <th>C</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Small</td>
    <td>450</td>
    <td>750</td>
    <td>950</td>
  </tr>
  <tr>
    <td>Medium</td>
    <td>350</td>
    <td>500</td>
    <td>800</td>
  </tr>
  <tr>
    <td>Large</td>
    <td>300</td>
    <td>450</td>
    <td>700</td>
  </tr>
  <tr>
    <td>Extra Large</td>
    <td>225</td>
    <td>300</td>
    <td>425</td>
  </tr>
</tbody>
</table>

Each machine is set up to work up to 50 hours per week and the hourly operating cost of these machines are respectively 40.00 USD, 60.00 USD, and 90.00 USD. Further suppose that 13000, 8000, 8000, and 9000 meters are the weekly production requirement for the product sizes: small, medium, large and extra large, respectively. Formulate the machine scheduling problem as a linear program and code it on Python/Julia.

## Mathematical modelling

### Decision variable:
$x_{ij}$: amount of working hour at machine $j$ for product size $i$, for $i=1, \dots ,4$ and $j=1,2,3$

### Parameters:
$p_{ij}$: productivity of machine $j$ for product size $i$ (feet/hour)

$c_j$: operating cost of machine $j$ ($/hour)

$d_i$: demand of product size $i$ (feet)

### Formulation:
$\min z = \sum_{j=1}^3 \sum_{i=1}^4 c_jx_{ij}$

Subject to:

$\sum_{i=1}^4 x_{ij} \leq 50, ~~ \forall j=1,2,3$ (Maximum operating time of machine $j$)

$\sum_{j=1}^3 a_{ij} x_{ij} = d_i, ~~ \forall i = 1, \dots, 4$ (Demand of beam size $i$)

$x_{ij} \geq 0, ~~\forall i=1,2,3 ~\text{and}~ j=1,\dots,4$ (Sign restriction)

## Computational modelling

In [1]:
# Check installed solvers
import cvxpy as cvx
print(cvx.installed_solvers())

['CPLEX', 'ECOS', 'ECOS_BB', 'GUROBI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS', 'XPRESS']


In [2]:
# Import packages
import cvxpy as cvx
import pandas as pan
import math
import numpy as np

In [3]:
# Productivity data:
p = np.array([[450, 750, 950],
              [350, 400, 800],
              [300, 350, 700],
              [225, 300, 425]])
# Cost data:
c = np.array([40, 60, 90])

# Demand data:
d = np.array([13000, 8000, 8000, 9000])

In [4]:
# Python indexing starts from 0
# Set declaration:
N = 4 # Number of product sizes
M = 3 # Number of machines

# Constraints:
constr = []

# Variables:
x = cvx.Variable((N, M), nonneg=True)

# Constraints:
## Maximum operating time of machine j
for j in range(M):
    constr += [sum(x[i,j] for i in range(N)) <= 50]

## Demand satisfaction
for i in range(N):
    constr += [sum(p[i][j]*x[i,j] for j in range(M)) == d[i]]
    
# Objective function:
objf = sum(sum(c[j]*x[i,j] for i in range(N)) for j in range(M))

# Call the solver:
prob = cvx.Problem(cvx.Minimize(objf), constr)
prob.solve(solver = 'ECOS_BB', verbose=True, warm_start=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 08 11:05:35 PM: Your problem has 12 variables, 7 constraints, and 0 parameters.
(CVXPY) May 08 11:05:35 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 08 11:05:35 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 08 11:05:35 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 08 11:05:35 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) May 08 11:05:35 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing

4568.571429215183

## Results

In [5]:
for i in range(N):
    for j in range(M):
        print('Product size ',i+1,' produced in machine ',j+1,' :',x[i,j].value)

Product size  1  produced in machine  1  : 2.063718818640068e-08
Product size  1  produced in machine  2  : 17.33333331244499
Product size  1  produced in machine  3  : 6.715289207511159e-09
Product size  2  produced in machine  1  : 7.854029435901991e-08
Product size  2  produced in machine  2  : 6.078157720282934e-09
Product size  2  produced in machine  3  : 9.999999962599542
Product size  3  produced in machine  1  : 3.61289704996184e-08
Product size  3  produced in machine  2  : 6.194677835249405e-09
Product size  3  produced in machine  3  : 11.428571409990248
Product size  4  produced in machine  1  : 39.99999997025554
Product size  4  produced in machine  2  : 1.276694114168213e-08
Product size  4  produced in machine  3  : 6.735117167079925e-09


---
# Question 2

## Problem description

A construction company has three jobsites that require 700, 900, and 800 tons of sand on weekly basis. The construction firm may purchase the sand from 3 different suppliers. The first two suppliers virtually have an unlimited supply and, because of other commitments, the third supplier cannot ship more than 600 tons weekly. The first supplier sign a transportation contract with a railway company, and there is no limit on the shipping capacity. On the other hand, the last two suppliers use trucks that limit the maximum shipping capacity up to 300 tons/shipment. The following table demonstrates the shipping cost from the suppliers to the jobsites of construction company (USD per ton)

<table>
<thead>
  <tr>
    <th></th>
    <th>Job site 1</th>
    <th>Job site 2</th>
    <th>Job site 3</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Supplier 1</td>
    <td><center>2</center></td>
    <td><center>4</center></td>
    <td><center>6</center></td>
  </tr>
  <tr>
    <td>Supplier 2</td>
    <td><center>4.5</center></td>
    <td><center>5</center></td>
    <td><center>5.8</center></td>
  </tr>
  <tr>
    <td>Supplier 3</td>
    <td><center>4.5</center></td>
    <td><center>4.6</center></td>
    <td><center>4.2</center></td>
  </tr>
</tbody>
</table>

Formulate the machine scheduling problem as a linear program and code it on Python/Julia.

## Mathematical modelling

### Decision variable:
$x_{ij}$: amount of materials which jobsite $j$ buy from supplier $i$, for $i=1, \dots ,3$ and $j=1,\dots ,3$

### Parameters:
$c_{ij}$: cost per unit of materials which jobsite $j$ buy from supplier $i$ (USD per ton)

$d_j$: demand of job site $j$ (ton)

### Formulation:
$\min z = \sum_{j=1}^3 \sum_{i=1}^3 c_{ij} x_{ij}$

Subject to:

$\sum_{j=1}^3 x_{3j} \leq 600$ (Upper limit of supply by the 3rd supplier)

$\sum_{i=1}^3 x_{ij} = d_j, ~~ \forall j = 1,2,3$ (Demand of each jobsite $j$)

$x_{ij} \leq 300, ~~ \forall i=2,3, ~~ \forall j=1,2,3$ (Upper limit of supply per shipment by the 2nd and 3rd suppliers)

$x_{ij} \geq 0, ~~ \forall i=1,2,3 ~\text{and}~ j=1,2,3$ (Sign restriction)

## Computational modelling

In [6]:
# Check installed solvers
import cvxpy as cvx
print(cvx.installed_solvers())

['CPLEX', 'ECOS', 'ECOS_BB', 'GUROBI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS', 'XPRESS']


In [7]:
# Import packages
import cvxpy as cvx
import pandas as pan
import math
import numpy as np

In [8]:
# Cost data:
c = np.array([[2,   4,   6],
              [4.5, 5,   5.8],
              [4.5, 4.6, 4.2]])

# Demand data:
d = np.array([700, 900, 800])

In [9]:
# Python indexing starts from 0
# Set declaration:
N = 3 # Number of suppliers
M = 3 # Number of jobsites

# Constraints:
constr = []

# Variables:
x = cvx.Variable((N, M), nonneg=True)

# Constraints:
## Upper limit of supply by the 3rd supplier
constr += [sum(x[2,j] for j in range(M)) <= 600]

## Demand satisfaction at each jobsite
for j in range(M):
    constr += [sum(x[i,j] for i in range(N)) == d[j]]
    
## Upper limit of supply per shipment by the 2nd and 3rd suppliers
for i in range(N):
    for j in range(1,2):
        constr += [x[i,j] <= 300]
    
# Objective function:
objf = sum(sum(c[i,j]*x[i,j] for i in range(N)) for j in range(M))

# Call the solver:
prob = cvx.Problem(cvx.Minimize(objf), constr)
prob.solve(solver = 'ECOS_BB', verbose=True, warm_start=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 08 11:05:47 PM: Your problem has 9 variables, 7 constraints, and 0 parameters.
(CVXPY) May 08 11:05:47 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 08 11:05:47 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 08 11:05:47 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 08 11:05:47 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) May 08 11:05:47 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

9640.00000253433

---
# Question 3

## Problem description

A coffee shop is open Monday-Friday from 9am to 5pm. From past experience, the coffee shop knows that it needs the number of staffs shown in the table below.

<table>
<thead>
  <tr>
    <th>Time Period</th>
    <th>Number of staffs<br>Required</th>
    <th>Time Period</th>
    <th>Number of staffs<br>Required</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td><center>09:00 - 10:00</center></td>
    <td><center>4</center></td>
    <td><center>13:00 - 14:00</center></td>
    <td><center>5</center></td>
  </tr>
  <tr>
    <td><center>10:00 - 11:00</center></td>
    <td><center>3</center></td>
    <td><center>14:00 - 15:00</center></td>
    <td><center>6</center></td>
  </tr>
  <tr>
    <td><center>11:00 - 12:00</center></td>
    <td><center>4</center></td>
    <td><center>15:00 - 16:00</center></td>
    <td><center>8</center></td>
  </tr>
  <tr>
    <td><center>12:00 - 13:00</center></td>
    <td><center>6</center></td>
    <td><center>16:00 - 17:00</center></td>
    <td><center>10</center></td>
  </tr>
</tbody>
</table>


The coffee shop hires two types of staffs: full-time and part-time. 
Full-time staffs work from 9:00 to 17:00 five days a week, including 1 hour off for lunch. 
The coffee shop determines the lunch hour of each full-time staff, but each staff must go for lunch either 12:00–13:00 or 13:00–14:00. 
Full-time staffs are paid 15 USD/hour (including lunch hour). 
The coffee shop may also hire part-time staffs. 
Each parttime staff must work exactly 4 consecutive hours each day. 
A part-time staff is paid 10 USD/hour. 
To maintain adequate quality of service, the coffee shop has decided that at most 5 part-time staffs can be hired. 
Formulate an Linear Program and code it on Python/Julia to meet the staff requirements at minimum cost. 

## Mathematical modelling

### Summary:
Full-time staffs: 9:00 – 17:00 (either 12:00 - 13:00 or 13:00 - 14:00 for lunch), 8 USD/hour

Part-time staffs: 4 hours/day, no more than 5 staffs can be hired, 5 USD/hour

### Decision variable:
$x_1$: number of full-time staffs taking 12PM-1PM time slot for lunch break

$x_2$: number of full-time staffs taking 1PM-2PM time slot for lunch break

$y_1$: number of part-time staffs working between 09:00 – 13:00

$y_2$: number of part-time staffs working between 10:00 – 14:00

$y_3$: number of part-time staffs working between 11:00 – 15:00

$y_4$: number of part-time staffs working between 12:00 – 16:00

$y_5$: number of part-time staffs working between 13:00 – 17:00

### Parameters:

$c_1$ is the Pay rate of full-time staffs: 15×8=120 USD/day

$c_2$ is the Pay rate of part-time staffs: 10x4=40 USD/day

### Formulation:
$\min z = 120(x_1 + x_2) + 40(y_1 + y_2 + y_3 + y_4)$

Subject to:

$x_1 + x_2 + y_1 \geq 4$ (Time period: 09:00 – 10:00)

$x_1 + x_2 + y_1 + y_2 \geq 3$ (Time period: 10:00 – 11:00)

$x_1 + x_2 + y_1 + y_2 + y_3 \geq 4$ (Time period: 11:00 – 12:00)

$x_1 + y_1 + y_2 + y_3 + y_4 \geq 6$ (Time period: 12:00 – 13:00)

$x_2 + y_2 + y_3 + y_4 + y_5 \geq 5$ (Time period: 13:00 – 14:00)

$x_1 + x_2 + y_3 + y_4 + y_5 \geq 6$ (Time period: 14:00 – 15:00)

$x_1 + x_2 + y_4 + y_5 \geq 8$ (Time period: 15:00 – 16:00)

$x_1 + x_2 + y_5 \geq 10$ (Time period: 16:00 – 17:00)

$y_1 + y_2 + y_3 + y_4 + y_5 \leq 5$ (Limited headcount of part-time staffs)

$x_1, x_2, y_1, y_2, y_3, y_4, y_5 \geq 0$ (Sign constraint)

## Computational modelling

In [10]:
# Check installed solvers
import cvxpy as cvx
print(cvx.installed_solvers())

['CPLEX', 'ECOS', 'ECOS_BB', 'GUROBI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS', 'XPRESS']


In [11]:
# Import packages
import cvxpy as cvx
import pandas as pan
import math
import numpy as np

In [12]:
# Cost data:
c1 = 15*8 #USD/shift
c2 = 10*4 #USD/shift

In [13]:
# Python indexing starts from 0
# Set declaration:
N = 2 # Number of shifts for full-time staffs
M = 5 # Number of shifts for part-time staffs

# Constraints:
constr = []

# Variables:
x = cvx.Variable(N, nonneg=True)
y = cvx.Variable(M, nonneg=True)

# Constraints:

## Time period: 09:00 – 10:00:
constr += [x[0] + x[1] + y[0] >= 4]

## Time period: 10:00 – 11:00:
constr += [x[0] + x[1] + y[0] + y[1] >= 3]

## Time period: 11:00 – 12:00:
constr += [x[0] + x[1] + y[0] + y[1] + y[2] >= 4]

## Time period: 12:00 – 13:00:
constr += [x[0] + y[0] + y[1] + y[2] + y[3] >= 6]

## Time period: 13:00 – 14:00:
constr += [x[1] + y[1] + y[2] + y[3] + y[4] >= 5]

## Time period: 14:00 – 15:00:
constr += [x[0] + x[1] + y[2] + y[3] + y[4] >= 6]

## Time period: 15:00 – 16:00:
constr += [x[0] + x[1] + y[3] + y[4] >= 8]

## Time period: 16:00 – 17:00:
constr += [x[0] + x[1] + y[4] >= 10]

## Limited headcount of part-time staffs
constr += [y[0] + y[1] + y[2] + y[3] + y[4] <= 5]

# Objective function:
objf = c1*sum(x[i] for i in range(N)) + c2*sum(y[j] for j in range(M))

# Call the solver:
prob = cvx.Problem(cvx.Minimize(objf), constr)
prob.solve(solver = 'ECOS_BB', verbose=True, warm_start=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 08 11:05:56 PM: Your problem has 7 variables, 9 constraints, and 0 parameters.
(CVXPY) May 08 11:05:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 08 11:05:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 08 11:05:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 08 11:05:56 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) May 08 11:05:56 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

859.9999999507766

## Result

In [14]:
print('Time Period \t PT1 \t PT2 \t FT')
print('-------------------------------------')
print('09:00 - 10:00 \t', x[0].value, '\t', x[1].value, '\t', y[0].value)
print('-------------------------------------')
print('10:00 - 11:00 \t', x[0].value, '\t', x[1].value, '\t', y[0].value + y[1].value)
print('-------------------------------------')
print('11:00 - 12:00 \t', x[0].value, '\t', x[1].value, '\t', y[0].value + y[1].value + y[2].value)
print('-------------------------------------')
print('12:00 - 13:00 \t', x[0].value, '\t', x[1].value, '\t', y[0].value + y[1].value + y[2].value + y[3].value)
print('-------------------------------------')
print('13:00 - 14:00 \t', x[0].value, '\t', x[1].value, '\t', y[1].value + y[2].value + y[3].value + y[4].value)
print('-------------------------------------')
print('14:00 - 15:00 \t', x[0].value, '\t', x[1].value, '\t', y[2].value + y[3].value + y[4].value)
print('-------------------------------------')
print('15:00 - 16:00 \t', x[0].value, '\t', x[1].value, '\t', y[3].value + y[4].value)
print('-------------------------------------')
print('16:00 - 17:00 \t', x[0].value, '\t', x[1].value, '\t', y[4].value)

Time Period 	 PT1 	 PT2 	 FT
-------------------------------------
09:00 - 10:00 	 5.4999999993673665 	 0.0 	 0.0
-------------------------------------
10:00 - 11:00 	 5.4999999993673665 	 0.0 	 0.14837092724078657
-------------------------------------
11:00 - 12:00 	 5.4999999993673665 	 0.0 	 0.31539146028495524
-------------------------------------
12:00 - 13:00 	 5.4999999993673665 	 0.0 	 0.5000000002150838
-------------------------------------
13:00 - 14:00 	 5.4999999993673665 	 0.0 	 5.000000000667313
-------------------------------------
14:00 - 15:00 	 5.4999999993673665 	 0.0 	 4.851629073426526
-------------------------------------
15:00 - 16:00 	 5.4999999993673665 	 0.0 	 4.684608540382358
-------------------------------------
16:00 - 17:00 	 5.4999999993673665 	 0.0 	 4.500000000452229
